
Siamese network for speaker verification


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import math
import librosa
import pickle
import tensorflow as tf
import os

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Change the path for the required input files

In [0]:
path = './gdrive/My Drive/hw4/'

path_result = path + 'result/'
path_model = path + 'model/'

In [0]:
def load_wav(file,f=0):
  sn, sr=librosa.load(file, sr=None)
  Xdata1=librosa.stft(sn, n_fft=1024, hop_length=512)
  if f==1: return sn,sr,Xdata1.T
  return Xdata1.T

In [0]:
import pickle
with open(path+'hw4_trs.pkl', 'rb') as f:
  loaded_train = pickle.load(f)
with open(path+'hw4_tes.pkl', 'rb') as f:
  loaded_test = pickle.load(f)  


In [0]:
import itertools,random

random.seed(123)
tf.random.set_random_seed(123)


Train and test files are converted into stft, and then stored as dictionary for faster access

In [0]:
trainset = {i:np.abs(librosa.stft(loaded_train[i], n_fft=1024, hop_length=512)).T for i in range(loaded_train.shape[0])}
testset = {i:(np.abs(librosa.stft(loaded_test[i], n_fft=1024, hop_length=512)).T) for i in range(loaded_test.shape[0])}

In [0]:
testset[1].shape

(45, 513)

In [0]:
def rand_l(num,l):
  random.shuffle(num)
  return num[:l]

def get_Lindices(num_range,L):
  num = list(itertools.combinations(num_range, 2))
  return rand_l(num,L)

def make_batch(i,L,inp):
  maxl = len(inp.keys())
  r1 = range(i*10,i*10+10)
  #print(maxl,r1)
  true_pairs = get_Lindices(r1,L)
  l1 = int((i*10/(maxl-(1)*10)) * L) if i!=49 else L
  #print(l1)
  num1 = [j for j in range(i*10)]
  num2 = [j for j in range(i*10+10,maxl)]
  num = rand_l(num1,l1)+ rand_l(num2,L-l1)
  false_pairs = [(random.choice(r1),tmp) for tmp in num]
  
  batch  = [(inp[a],inp[b],1) for a,b in true_pairs]
  [batch.append((inp[a],inp[b],0)) for a,b in false_pairs]
  #print(len(true_pairs),len(false_pairs),len(batch))
  random.shuffle(batch)
  return batch
def get_batches(inp,L):
  maxl = int(len(inp.keys())/10)
  #L= 45 #randint(15,40)
  return [make_batch(i,L,inp) for i in range(maxl)]
  

In [0]:
#batch = make_batch(18,25,train)
batches = get_batches(trainset,25)
testbatch = get_batches(testset,45)
#batches
#print(len(batches[2]))
#print(range(2))

In [0]:
n_features = 513
n_col = 32 

# placeholder for the label. `1` for `same` and `0` for `different`.

# placeholder for dropout (we could use different dropout for different part of the architecture)
keep_prob = tf.placeholder(tf.float32)
timestep = tf.placeholder(tf.int32)

In [0]:

num_hidden = 513
latent_length = 16
initconv = tf.contrib.layers.variance_scaling_initializer(uniform=True,factor=1.0)

def network(X,re=False):
  #lstm = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_hidden, initializer=initconv,reuse=re,name="lstm1"),output_keep_prob=keep_prob)#,reuse=tf.AUTO_REUSE)
  rnn_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(int(num_hidden),name="gru"+str(i),reuse=re,kernel_initializer = initconv),output_keep_prob=keep_prob) for i in range(2)])
  output, state = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)
  
  dense = tf.layers.dense(inputs=output,units=8*latent_length, activation=tf.nn.tanh,kernel_initializer=initconv,bias_initializer=initconv,reuse=re,name="dense1")#,reuse=tf.AUTO_REUSE)
  dropout = tf.layers.dropout(inputs=dense, rate=1-keep_prob)
  dense2 = tf.layers.dense(inputs=dense,units=latent_length, activation=tf.nn.tanh,kernel_initializer=initconv,bias_initializer=initconv,reuse=re,name="dense2")
  flat = tf.reshape(dense2,[-1,timestep*latent_length])
  return flat


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
size = tf.placeholder(tf.int64)
x1 = tf.placeholder(tf.float32, shape=[None, None, n_features]) # placeholder for the first network (image 1)
x2 = tf.placeholder(tf.float32, shape=[None, None, n_features]) # placeholder for the second network (image 2)
y = tf.placeholder(tf.float32)
learning_rate = 0.0002


#output_1 = tf.Variable((n_units),dtype= tf.float32)
#output_2 = tf.Variable((n_units),dtype= tf.float32)

output_1 = network(x1,re=False)
output_2 = network(x2,re=True)
  
#sig =  tf.sigmoid(tf.reduce_sum(output_1*output_2))
#loss = -(y)*tf.log(sig+ (10**-6)) - (1 - y) * tf.log(1 - sig + (10**-6))

sig =  tf.reduce_sum(output_1*output_2,axis=1)
loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=sig))


train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss=loss)

sess = tf.Session()
saver = tf.train.Saver()
init = tf.global_variables_initializer()


Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use tf.cast instead.


In [0]:
import math

def sigmoid(x):
  return 1 / (1 + np.exp(-np.array(x)))

In [0]:
def network_initializer(self,inp1,inp2,ytrue):
  global output_1,ouput_2
  
    
sess.run(tf.global_variables_initializer())
num_epochs = 150
for epoch in range(num_epochs):
  loss_tot = 0
  i=0
  batches = get_batches(trainset,30)
  for batch in batches:
    a,b,yt = [np.array(list(t)) for t in zip(*batch)]
    time = a[0].shape[0]
    _, lossi = sess.run([train, loss], feed_dict={x1: a,x2: b, y: yt,timestep: time,keep_prob: 0.9})
    loss_tot += lossi
    i+=1    
    #print("batch over ",i)  
  print("epoch loss",epoch,loss_tot)    

pred = []
ytrue = []  
for batch in testbatch:
    a,b,yt = [np.array(list(t)) for t in zip(*batch)]
    time = a[0].shape[0]
    _, dotp = sess.run([train, sig], feed_dict={x1: a,x2: b, y: yt,timestep: time,keep_prob: 1})
    predy = sigmoid(dotp)
    pred.append(predy)
    ytrue.append(yt)
pred = np.array(pred)
#pred = 1*(pred>0.5)
ytrue = np.array(ytrue)
#print("test accuracy: ",((pred == ytrue).sum())/len(pred) )
saver.save(sess, path_model + 'model_lstm.ckpt')
    
    
#_, lossi = sess.run([train, loss], feed_dict={X: batch_x, Y: batch_y,keep_prob: 0.9})

    


epoch loss 0 13778.723754882812
epoch loss 1 6314.522705078125
epoch loss 2 4057.3130264282227
epoch loss 3 2806.2255821228027
epoch loss 4 2246.533847808838
epoch loss 5 1955.5712928771973
epoch loss 6 1946.0500144958496
epoch loss 7 1900.1036014556885
epoch loss 8 1832.932378768921
epoch loss 9 1809.6793804168701
epoch loss 10 1690.5558853149414
epoch loss 11 1722.602373123169
epoch loss 12 1747.4804611206055
epoch loss 13 1719.8602867126465
epoch loss 14 1645.0672149658203
epoch loss 15 1667.925428390503
epoch loss 16 1630.6245250701904
epoch loss 17 1690.403709411621
epoch loss 18 1592.2060260772705
epoch loss 19 1630.7407894134521
epoch loss 20 1587.1397686004639
epoch loss 21 1553.7004985809326
epoch loss 22 1607.2493152618408
epoch loss 23 1612.748649597168
epoch loss 24 1643.2827701568604
epoch loss 25 1551.5123310089111
epoch loss 26 1591.4881343841553
epoch loss 27 1545.7445850372314
epoch loss 28 1582.0485610961914
epoch loss 29 1558.999402999878
epoch loss 30 1536.777383804

'./gdrive/My Drive/hw4/model/model_lstm.ckpt'

In [0]:
pred2 = 1*(pred>0.5)
ytrue = np.array(ytrue)
print("test accuracy: ",((pred2 == ytrue).sum())/(len(pred)*pred2.shape[1]) )


test accuracy:  0.6727777777777778


In [0]:
np.array(testbatch).shape

(20, 90, 3)

In [0]:
pred

array([[0.47507143, 0.89520216, 0.7364979 , ..., 0.87916416, 0.2852581 ,
        0.93710196],
       [0.0369285 , 0.9246945 , 0.993551  , ..., 0.9875888 , 0.9804166 ,
        0.9847685 ],
       [0.58708084, 0.67255265, 0.4650819 , ..., 0.7056815 , 0.99227303,
        0.36453125],
       ...,
       [0.886155  , 0.69482344, 0.4457068 , ..., 0.40855157, 0.9747799 ,
        0.46776158],
       [0.87341374, 0.87005377, 0.89092505, ..., 0.40987608, 0.9534089 ,
        0.9307398 ],
       [0.37257528, 0.6927203 , 0.94164133, ..., 0.62666994, 0.6767964 ,
        0.28169632]], dtype=float32)

https://github.com/ardiya/siamesenetwork-tensorflow/blob/master/train.py